In [13]:
import lasio
import pandas as pd
import numpy as np

def generate_facies_curves_to_las(las_file, output_las):
    """
    Generate facies curves from probability logs in a LAS file and save into a new LAS.

    Facies assignment rule:
        facies = 1 where probability is max among available probability logs
        (Sand, Silt, Shale, Limestone, Dolomite — whichever exist)

    Creates an additional single-code lithology curve:
        0 = FACIESSHALE
        1 = FACIESSILT
        2 = FACIESSANDSTONE
        3 = FACIESLIMESTONE
        4 = FACIESDOLOMITE

    If a probability curve is missing, it is simply ignored.
    """

    # Load LAS
    las = lasio.read(las_file)

    # Convert to DataFrame (depth becomes a normal column)
    df = las.df().reset_index()  # DEPT becomes a column

    # Preferred order (used to break ties deterministically)
    preferred_order = ["SAND_PROB", "SILT_PROB", "SHALE_PROB", "LIMESTONE_PROB", "DOLOMITE_PROB"]
    prob_cols = [c for c in preferred_order if c in df.columns]

    if not prob_cols:
        raise ValueError("No probability curves found in LAS file.")

    # Initialize facies columns (0 default, only for available curves)
    facies_map = {
        "SAND_PROB": "FaciesSand",
        "SILT_PROB": "FaciesSilt",
        "SHALE_PROB": "FaciesShale",
        "LIMESTONE_PROB": "FaciesLimestone",
        "DOLOMITE_PROB": "FaciesDolomite"
    }

    for prob_col in prob_cols:
        df[facies_map[prob_col]] = 0

    # Compute max-probability column for each row
    max_probs = df[prob_cols].idxmax(axis=1)

    # Assign binary facies
    for prob_col in prob_cols:
        facies_col = facies_map[prob_col]
        df.loc[max_probs == prob_col, facies_col] = 1

    # Map the prob column name to lithology code
    mapping = {
        "SHALE_PROB": 0,
        "SILT_PROB": 1,
        "SAND_PROB": 2,
        "LIMESTONE_PROB": 3,
        "DOLOMITE_PROB": 4
    }

    # Create LITHOLOGY curve
    df["LITHOLOGY"] = max_probs.map(mapping).astype(float)

    # Append facies curves back into LAS
    for prob_col in prob_cols:
        facies_col = facies_map[prob_col]
        las.append_curve(facies_col, df[facies_col].values, descr=f"{facies_col} curve (0/1)")

    # Append single lithology curve
    lith_descr = "FACIESSHALE: 0, FACIESSILT: 1, FACIESSANDSTONE: 2, FACIESLIMESTONE: 3, FACIESDOLOMITE: 4"
    las.append_curve("LITHOLOGY", df["LITHOLOGY"].values, descr=lith_descr)

    # Write to output LAS
    las.write(output_las, version=2.0, wrap=False)
    print(f"Facies and lithology curves written to: {output_las}")


In [16]:
generate_facies_curves_to_las('HIKMAT-1.las', 'HIKMAT-1_Lithology.las')

C:\Users\HBSohail\AppData\Local\Temp\ipykernel_1720\2194379673.py:49: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  max_probs = df[prob_cols].idxmax(axis=1)


Facies and lithology curves written to: HIKMAT-1_Lithology.las
